In [10]:
import os
from collections import OrderedDict
import nltk
nltk.download('punkt')
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fashn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
corpus_path = 'Corpus 2'
stopwords_file = 'Stopwords.txt'

In [12]:

files=os.path.join(corpus_path)

# getting list of all files
corpus_files = os.listdir(corpus_path)
print(corpus_files)

documents=[]
queries_files = []

for file in corpus_files:
    if 'Document' in file:
        documents.append(file)
    elif 'Query' in file:
        queries_files.append(file)
        

documents=sorted(documents, key=lambda x: int(re.findall('\d+', x)[0]))
queries_files=sorted(queries_files, key=lambda x: int(re.findall('\d+', x)[0]))
print(documents)
print(queries_files)


['Document 1.txt', 'Document 10.txt', 'Document 2.txt', 'Document 3.txt', 'Document 4.txt', 'Document 5.txt', 'Document 6.txt', 'Document 7.txt', 'Document 8.txt', 'Document 9.txt', 'Query 1.txt', 'Query 2.txt', 'Query 3.txt', 'Query 4.txt']
['Document 1.txt', 'Document 2.txt', 'Document 3.txt', 'Document 4.txt', 'Document 5.txt', 'Document 6.txt', 'Document 7.txt', 'Document 8.txt', 'Document 9.txt', 'Document 10.txt']
['Query 1.txt', 'Query 2.txt', 'Query 3.txt', 'Query 4.txt']


In [13]:
# Load stopwords
with open(stopwords_file, 'r') as f:
    stopwords = set(f.read().splitlines())

print(stopwords)

{'and', 'at', 'that', 'down', 'under', 'such', 'few', 'on', 'or', 'this', 'theirs', 'will', 'by', 'who', 'all', 'what', 'be', 'off', 'i', 'each', 'an', 'only', 'those', 'there', 'until', 'should', 'my', 'before', 'as', 'how', 'after', 'him', 'hers', 'we', 'myself', 'their', 'when', 'your', 'the', 'itself', 'whom', 'don', 'does', 'because', 'further', 'me', 'while', 'our', 'which', 'having', 'about', 'then', 'most', 'himself', 'once', 'her', 'no', 'just', 'she', 'between', 'in', 'against', 's', 'own', 'here', 'with', 'were', 'themselves', 'these', 'very', 'am', 'during', 'you', 'above', 'have', 'doing', 'same', 'of', 'both', 'other', 'can', 'they', 'not', 'are', 'for', 'he', 't', 'it', 'had', 'now', 'up', 'his', 'yourself', 'did', 'through', 'yours', 'do', 'has', 'below', 'from', 'a', 'being', 'is', 'some', 'if', 'out', 'to', 'where', 'so', 'but', 'its', 'herself', 'why', 'than', 'more', 'too', 'ourselves', 'was', 'yourselves', 'been', 'them', 'ours', 'over', 'nor', 'again', 'any', 'int

In [18]:
# Tokenize and preprocess documents
corpus = {}
terms = set()
for filename in documents:
    with open(os.path.join(corpus_path, filename), 'r', encoding='utf-8') as f:
        text = f.read()
        tokens = nltk.word_tokenize(text.lower())
        tokens = [token for token in tokens if token.isalpha() and token not in stopwords]
        porter_stemmer = nltk.PorterStemmer()
        stems = [porter_stemmer.stem(token) for token in tokens]
        corpus[filename]=stems
        for term in stems:
            terms.add(term)

#         print(stems)
        
        
print(corpus)

{'Document 1.txt': ['inform', 'retriev', 'favourit', 'program', 'languag', 'inform', 'retriev', 'program', 'assign', 'inform', 'retriev', 'one', 'kind', 'activ', 'main', 'goal', 'obtain', 'data', 'resourc', 'accord', 'inform', 'need', 'collect', 'inform', 'resourc'], 'Document 2.txt': ['inform', 'retriev', 'inform', 'retriev', 'perform', 'plan', 'inform', 'retriev', 'inform', 'retriev', 'techniqu', 'evalu', 'result', 'evalu', 'result', 'inform', 'retriev', 'step', 'inform', 'retriev'], 'Document 3.txt': ['inform', 'retriev', 'ir', 'comput', 'inform', 'scienc', 'process', 'obtain', 'inform', 'system', 'resourc', 'relev', 'inform'], 'Document 4.txt': ['manag', 'graduat', 'high', 'demand', 'across', 'industri', 'like', 'inform', 'technolog', 'design', 'studi', 'dynam', 'specialis', 'comput', 'scienc', 'better', 'comput', 'scienc', 'inform', 'technolog', 'thing', 'better', 'comput', 'scienc', 'inform', 'technolog', 'lead', 'differ', 'career', 'differ', 'specialis', 'subfield'], 'Document 5

In [19]:
# initializing dataframe
matrix = pd.DataFrame(terms,columns=['Terms'])
new = ['idf']
temp = 'tf-idf'
for i in range(len(documents)):
    new.append(temp + ' '+str(i+1))

matrix[documents] = pd.DataFrame([[None]*len(documents)], index=matrix.index)
matrix[new]=pd.DataFrame([[None]*len(new)], index=matrix.index)

matrix

,Terms,Document 1.txt,Document 2.txt,Document 3.txt,Document 4.txt,Document 5.txt,Document 6.txt,Document 7.txt,Document 8.txt,Document 9.txt,...,tf-idf 1,tf-idf 2,tf-idf 3,tf-idf 4,tf-idf 5,tf-idf 6,tf-idf 7,tf-idf 8,tf-idf 9,tf-idf 10
0,demand,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,prepar,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,plan,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,collect,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,human,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,degre,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
126,activ,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
127,techniqu,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
128,practition,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [20]:
import math

# print(len(documents))
# print(len(corpus[documents[0]]))

for idx, term in matrix['Terms'].iteritems():
    df = 0
    for doc in documents:
        if term in corpus[doc]:
            df+=1
#             tf calculation
            tf = corpus[doc].count(term) / len(corpus[doc])
#             matrix.loc[idx, doc] = tf
            if tf>0:
                matrix.loc[idx, doc] =1 + math.log10(tf)
            else:
                matrix.loc[idx, doc] = 0
        else:
            matrix.loc[idx, doc] = 0
    
#     idf
    idf = math.log10(len(documents)/df)
    matrix.loc[idx, 'idf'] = idf
    
#     tf-idf
    temp = 'tf-idf'
    i = 0
    for doc in documents:
        temp1 = temp + ' '+str(i+1)
        tf_idf = matrix.loc[idx,doc] * idf
        matrix.loc[idx, temp1]=tf_idf
        i+=1

# matrix = matrix.drop("tf-idf", axis='columns')
matrix.to_csv('tf-idf.csv', index=False)

In [21]:
unitVector = pd.DataFrame(terms,columns=['Terms'])
temp = 'unit-vector'
new= []
for i in range(len(documents)):
    new.append(temp + ' '+str(i+1))

unitVector[new]=pd.DataFrame([[None]*len(new)], index=unitVector.index)
unitVector

,Terms,unit-vector 1,unit-vector 2,unit-vector 3,unit-vector 4,unit-vector 5,unit-vector 6,unit-vector 7,unit-vector 8,unit-vector 9,unit-vector 10
0,demand,None,None,None,None,None,None,None,None,None,None
1,prepar,None,None,None,None,None,None,None,None,None,None
2,plan,None,None,None,None,None,None,None,None,None,None
3,collect,None,None,None,None,None,None,None,None,None,None
4,human,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
125,degre,None,None,None,None,None,None,None,None,None,None
126,activ,None,None,None,None,None,None,None,None,None,None
127,techniqu,None,None,None,None,None,None,None,None,None,None
128,practition,None,None,None,None,None,None,None,None,None,None


In [67]:
import math
temp1 = 'unit-vector'
name = 'tf-idf'
for i in range(len(documents)):
    new = name + ' '+str(i+1)
    temp = matrix[new].values.tolist()
    magnitude = math.sqrt(sum([x**2 for x in temp]))
    unit_vector = [x/magnitude for x in temp]
    new = temp1 + ' '+str(i+1)
    unitVector[new] = unit_vector
        
        
# matrix = matrix.drop("tf-idf", axis='columns')
unitVector.to_csv('unit-vector.csv', index=False)
unitVector

,Terms,unit-vector 1,unit-vector 2,unit-vector 3,unit-vector 4,unit-vector 5,unit-vector 6,unit-vector 7,unit-vector 8,unit-vector 9,unit-vector 10
0,demand,0.000000,0.000000,0.0,-0.287915,0.0,0.000000,0.000000,0.000000,0.000000,0.0
1,prepar,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,-0.230354,0.000000,0.0
2,plan,0.000000,-0.476568,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
3,collect,-0.281615,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
4,human,0.000000,0.000000,0.0,0.000000,0.0,0.000000,-0.194431,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
125,degre,0.000000,0.000000,0.0,0.000000,0.0,-0.396227,0.000000,0.000000,0.000000,0.0
126,activ,-0.281615,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
127,techniqu,0.000000,-0.333107,0.0,0.000000,0.0,0.000000,-0.135902,0.000000,0.000000,0.0
128,practition,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,-0.239021,0.0
